# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pickle
import re
import pandas as pd # checked
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer #check
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report # checked
from sklearn.multioutput import MultiOutputClassifier #checked
from sklearn.ensemble import RandomForestClassifier # Checked
from sklearn.model_selection import GridSearchCV #checked
import nltk # checked
from nltk.corpus import stopwords # checked
from nltk.tokenize import word_tokenize # checked
from nltk.stem import WordNetLemmatizer

nltk.download(['punkt', 'stopwords', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterData.db')
df = pd.read_sql_table('DisasterData', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

In [3]:
print(X.head())
print(Y.head())

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object
   related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone      ...        \
0                  0         0         0            0      ...         
1                  0         0         0            0      ...         
2                  0         

### 2. Write a tokenization function to process your text data

In [4]:

def tokenize(text):    
    # URL regex pattern
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # Replace URLs with a space
    text = re.sub(url_regex, 'urlplaceholder', text)
    

    # Normalize by converting to lowercase and removing punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()
    
     #Split text into words using NLTK, Tokenize the text
    tokens = word_tokenize(text)
#     print(tokens)
    
    # Remove stopwords
    tokens = [tk for tk in tokens if tk not in stopwords.words("english")]
    
    # Lemmatize each token
    tokens = [lemmatizer.lemmatize(t) for t in tokens]

    return tokens

In [5]:
text = "Check out this link: https://example.com and another http://example2.com! It's amazing."

print(tokenize(text))

['check', 'link', 'urlplaceholder', 'another', 'urlplaceholder', 'amazing']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=8)

# Train pipeline
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# Test model
Y_pred = pipeline.predict(X_test)

# Print classification report for each category
for i, column in enumerate(Y_test.columns):
    print(f"Category: {column}")
    print(classification_report(Y_test[column], Y_pred[:, i]))

Category: related
             precision    recall  f1-score   support

          0       0.66      0.49      0.56      1251
          1       0.85      0.92      0.88      3955
          2       0.28      0.29      0.28        38

avg / total       0.80      0.81      0.80      5244

Category: request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      4364
          1       0.77      0.43      0.55       880

avg / total       0.87      0.88      0.87      5244

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5224
          1       0.00      0.00      0.00        20

avg / total       0.99      1.00      0.99      5244

Category: aid_related
             precision    recall  f1-score   support

          0       0.74      0.86      0.79      3060
          1       0.74      0.58      0.65      2184

avg / total       0.74      0.74      0.74      5244

Category: med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Define parameters for GridSearchCV
parameters = {
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 4],
#     'clf__estimator__max_features': ['auto', 'sqrt', 'log2']
}

# Do grid search
cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Test the tuned model
Y_pred_tuned = cv.predict(X_test)

# Print classification report for each category after tuning
for i, column in enumerate(Y_test.columns):
    print(f"Category: {column}")
    print(classification_report(Y_test[column], Y_pred_tuned[:, i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
# Export model as a pickle file
with open('DisasterData_classifier.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.